# Introduction

- In this notebook, i will be giving a comprehensive introduction and implementation to how CNN is implemented using only basic libraries such as numpy and cupy.

- CNN performs very well in image related tasks, such as image recognition, object detection.

- This notebook can be served as teaching and learning purpose, most of the code here will be the basis of my future research in computer vision and convolutional neural network. Lets take a look on the details of convolutional neural network first !

# Key terms
1. Feature : **Features are some characteristics in training data which they help doing the task** .We believe machine learns features from training data to finish the task very accurately, such as textures, colours, edges from images. Extracting features is an important goal of machine learning, by extracting feataures machine can do the assigned task very accurately. In CNN, feature maps usually mean the matrix after convolution and max-pooling. Which they contains features, some we may understand how they help doing the task, but mostly, we dont.

2. Learning Rate(Or step size) : When we are doing gradient descent optimization, we will calculate the gradient of loss against weight, then update the weight, step size measures **how big the step will be**, which is how the name "step size" come.

3. Bias : In machine learning, bias means the model has **bias or prejudice** to the input data, which might leads to inaccuarate prediction.

# Inspiration of CNN
- We can dive deeper into how human eyes work before we introduce CNN. In accordance to Hubel and Wiesel's (1959) research, they found that the visual cortex has small regions of cells that are sensitive to specific regions in the visual field. Which suggested that our eye might **not** be attentive to everywhere in our visual field.

- Hence, this inspires how can we make a model to process images, we first make a **kernel**, which its dimension is small compared to the whole image, then we slides the kernel in the image to get some features in the image to simulate how our eyes catch the light details of an object. Then we can send these "signals"(feature maps in machine learning) to our brain for further processings, such as identification and classification.
  

**1. Convolution**
- Convolution serves a major part in convolutional neural network, it is the "soul" of CNN, lets start from its mathematical definition, the conovlution of 2 functions f(t) and g(t) is defined as :

  $$f(t) \ast g(t) = \int^{\infty}_{-\infty} f(\tau)g(t-\tau) d\tau$$

The above definition can be generalized to grid-structured data such as matrix, in fact, in CNN, we use matrix to represent data more frequently, hence, we are able to define convolution for matrix, which is **dot product** of 2 matrices, because the convolution formula becomes :

$$Y(i,j) = \sum^{h-1}_{m=0} \sum^{w-1}_{n=1} X(i+m, j+n)\cdot W(m,n)$$

Which is a element-wise dot product.


**2. Image representation**
- How do we represent images in computer? There are 3 major components in image representation, which are **channels, height, width**. Height and Width are very easy to understand, what about channels?

- Well, we know that the most basic part in an image is **pixel**, **channel can be imagined as how many layers of rectangles you stacked where all these layers of rectangles have the same height and width**.

- For example, if channel = 1, it is simply grayscale image. If channel=3, it is RGB images, where you have 3 layers of rectangles here, Red, Green and Blue one. Each layer has same dimension. More channels allow the feature map to store more features but also lead to tremendous increment on memory usage and computational requirement, hence increasing channel is a trade-off between computational efficiency and model representation power.

# > **CNN architecture**

# 1. **Convolution**

When we have received an input image, we will immediately do convolution on the images, so what is the details?

  (The following shapes representations followed by Numpy traditions on representing matrix shapes)

  Suppose the input image has a shape of **(channels, height, width)**, at first, we have to initialize our filters into shape of **(output_channels, input_channels, kernel_height, kernel_width)**, for those who love math, here's for you :

$$X \in \mathcal{R}^{H \times W\times C}$$
$$W \in \mathcal{R}^{W_h \times W_w \times C_{in} \times C_{out} \times N}$$

$C_{in}$ and $C_{out}$ means the **output and input channels of the feature map**

How about the values inside these matrices? We mostly use some popular initialization techniques, for CNN, obviously He Intialization is the best, it is proposed by He. K. Zhang et.al(2015), where the values are given by :

  $$w \sim \mathcal{U}(0, \sqrt{\frac{2}{n_l}})$$

The shape of the feature map after convolution is :

$$F_{q+1} = \frac{F_q-B_q+2P}{S} + 1$$

Note that it can only be applied to calculate width and height of feature map, we will explain how this formula is derived in later section.

And the mathematical formula for convolution is :

$$h^{q+1}_{rsp} = \sum^{B_q}_{i=1} \sum^{B_q}_{j=1} \sum^{d_{q+1}}_{k=1} w^{(p,q)}_{rsk}\cdot h^{q}_{rS+i-P, sS+j-P,k} = \sum_{(i,j,k) \in \mathcal{K}} w^{(p,q)}_{rsk}\cdot h^{q}_{rS+i-P, sS+j-P,k}$$

For most general case, batch is included :
$$ = \sum_{(i,j,k,n) \in \mathcal{K}} w^{(p,q)}_{rsk}\cdot h^{q}_{rS+i-P, sS+j-P,k,n}$$

It looks very complex, but let me explain them by part ! GIF 1 gives you a more simpler explanation by demonstrating it visually !

$F_q$ is the shape of feature map in qth layer, $B_q$ is the shape of kernel connecting between q and (q+1)th layer, just think of it as weights connecting 2 layers, as you use this kernel to convolve with feature map(simply doing dot product) in qth layer to get the feature map in (q+1)th layer. Here we got 2 more new variables, S and P, who are they? They are **strides and padding** !

* Strides : Stride means when we are sliding filter in the feature map, **how many grids do we slide when in 1 move**, usually if stride is larger, we use less memory and computational power as we perform less dot product between kernel and selected region.
  
The convolution is performed at position of :

  $$1, S+1, 2S+1, .....$$ for S>1 
  
* In the demonstration below, it used stride=2, which means the kernel will move 2 grid in each move.


* Padding : In the above examples of demonstration of Convolution, you may have discovered that, after the convolution, the feature map dimension will be smaller, that is the nature of convolution, but unfortunately, it is **undesirable**, because the reduce of size leads to **some loss of information**, so the goal of padding is to solve this issue, how do we make the shape between and before convolution the same? That's right, that's the add some numbers around the feature map border, usually the number is 0, by adding padding into feature map, we can retain the shape of feature map !


Now look back at the shape formula we wrote in the beginning :

$$F_{q+1} = \frac{F_q-B_q+2P}{S} + 1$$

$F_q - B_q$ is caused by dot product, **only one scalar value will be returned after the dot product**, we simply stack these values together to create a feature map !

Suppose we add padding with thickness $P$ in the border, then our height/width dimension is increased by $2P$(dimension increased P in each side), hence we have $+2P$.

$/S$ and the $+1$ due to stride, because convolution start in the first position of the feature map, as we move $S$ grids at each move, the required times of convolution is divided by $S$.

* **Little Conclusion of this part**:
  - Stride : How many grids your kernel moves in 1 convolution.
  - Padding : Thickness of zeros you add in the border of your feature map, pad=0 means no padding, in most libraries, "same" padding meaning that the spatial dimensions of feature map before and after convolution is the same, which suggested that it uses zero-padding to retain the shape, for "valid" padding, it simply equals to no use of padding. 

* **im2col method for convolution**
  
Next, we are going to introduce an important algorithm to calculate convolution in CNN, which is called im2col(image to columns) technique, it is proposed by *Chellapilla, K., Puri, S., & Simard, P. (2006)*, the basic idea is to turn the kernel and feature maps into **column matrix**, hence we can change the series of dot product into matrix multiplication, then we can use the optimized BLAS library to boost the convolution. 

1. Flatten the kernel
   
By the following command, we can flatten our kernel :

**np.reshape(kernel, (out_channel ,-1))**

2. Slides over the feature map

Next we have to let our original kernel to slides over the feature map like normal convolution, but instead of performing **dot product**, we are going to reshape the **selected region** into column matrix.

Define a region proposal as $R$, where it is matrix with shape :


After we finished this process, we can stack all the column matrix together, the number of rows indicate **how many times our kernel has slides over the feature map**.

3. Matrix Multiplication

In real situation, with consideration to the channels, our column matrix should be a 2D array, as well as kernel. Hence, in the final step, we simply multiply the flattened kernel with the column matrix to get the feature map !

Mathematically:
$$columns = \left\{ X_{\text{padded}}[:, :, i \cdot s : i \cdot s + HH, j \cdot s : j \cdot s + WW] \mid \forall N, \forall C, 0 \leq i < \text{out}_h, 0 \leq j < \text{out}_w \right\}.\text{reshape}(C\times HH\times WW, N*out_h*out_w)$$
Or you can understand as stacking columns together into a new matrix for all possible (i,j) position:
$$columns = [col_1, col_2,....,col_{C\times HH\times WW}]$$
output = $$W.\text{reshape}(OC, IC\times HH\times WW) \times columns$$
Which is much faster than:
$$ = \sum_{(i,j,k,n) \in \mathcal{K}} w^{(p,q)}_{rsk}\cdot h^{q}_{rS+i-P, sS+j-P,k,n}$$

due to the computing power of GPU and mitigation of explicit for loops 


Here is the implementation of im2col : 


In [ ]:
#Implementation of im2col and conv2d
#Number of rows = times of convolution
import cupy as cp 

def im2col_cupy(X_train, kernel, stride, pad):
    # Ensure the kernel and input have matching in_channels
    # Flatten the kernel
    k_flatten = cp.reshape(kernel, newshape=(kernel.shape[0], -1))

    # Calculate output shape with given padding and strides
    N, C, H, W = X_train.shape
    OC, IC, HH, WW = kernel.shape
    assert IC == C 
    #Input channel of kernel and channel of feature map input channel must be the same 

    out_h, out_w = (H-HH+2*pad)//stride + 1, (W-WW+2*pad)//stride + 1

    # Pad the input if necessary
    if pad > 0:
        X_padded = cp.pad(X_train, pad_width=[(0, 0), (0, 0), (pad, pad), (pad, pad)], mode='constant')
    else:
        X_padded = X_train

    # Generate column matrix using efficient CuPy operations
    columns = cp.array([
        X_padded[:, :, i*stride:i*stride+HH,j*stride:j*stride+WW]
        for i in range(0, out_h)
        for j in range(0, out_w)
    ])
    col = columns.reshape((C*HH*WW, N*out_h*out_w))
    columns_reshaped = columns.reshape((N, C, HH, WW, out_h, out_w))
    del columns 

    #Use safe_matmul, safe_matmul can divide the a, b into chunks to do matrix multiplication in case memory is insufficient
    try:
        a = cp.matmul(k_flatten, col).reshape((N,OC,out_h,out_w))
    except cp.cuda.memory.OutOfMemoryError:
        r = []
        chunks = 2
        arrays = cp.array([col[:, i*(col.shape[1])//chunks:(i+1)*(col.shape[1]//chunks)] for i in range(chunks)])
        #Transform into array with shape of (chunks, ....)
        for i in range(chunks):
            with cp.cuda.Device(0):
                m = cp.matmul(k_flatten, arrays[i, :, :])
            r.append(m)
            #Concatenate the small chunks together to make the original matrix 
        del arrays 
        a = cp.concatenate(a, axis=1).reshape((N, OC, out_h, out_w))
        
    return columns_reshaped, k_flatten, a
    #Note that i have integrated the function of convolution into this program so it can directly output convolved feature map, so we dont need another conv2d function
X = cp.random.randn(1000, 3, 30, 30)
W = cp.random.randn(100, 3, 3, 3)
im2col_cupy(X, W, stride=1, pad=1)    

Later we also need **col2im** in CNN backpropagation, here is an implementation of col2im, we are simply reversing the process !

Thus, studying the code yourself can boost your understanding on numpy libraries and matrices operations !

In [ ]:
def col2im(cols:cp.array, input, kernel, stride, pad):

    N, C, H, W = input.shape
    _, _, HH, WW = kernel.shape 
    out_h = (H+2*pad-HH)//stride + 1
    out_w = (W+2*pad-WW)//stride + 1
    cols = cols.reshape(N*C, HH*WW, out_h, out_w)
    X = cp.zeros((N,C,H+2*pad,W+2*pad))
    #Place an assertion to detect incompitable shapes early 

    for i in range(HH): 
        for j in range(WW): 
            X[:, :, i:i+stride*out_h:stride, j:j+stride*out_w:stride] += cols[:, i*WW+j, :, :].reshape(N, C, out_h, out_w)

    if pad>0:
        X = X[:, :, pad:-pad, pad:-pad]
    return X 

# 2. **Max-Pooling**
-  Max-Pooling is another important concept in CNN, for large feature maps, the memory and computational power it takes to run the convolution is tremendous. Aside from insufficient memory, as we put more layers in, running convolution takes up more time, that is undesirable.

- Max-Pooling is a method to reduce **feature maps dimension**, the basic idea is to make up a "pool", then select the maximum values in the pool to represent the whole pool, it can reduce the dimension of feature map very effectively.

Mathematically, it is equivalent to :
- $$ h^{\text{out}}_{n, c, p, q} = \max_{\substack{0 \le i < PH \\ 0 \le j < PW}} h^{\text{in}}_{n, c, S \cdot p + i, S \cdot q + j} $$

- Same as convolution, we have 2 important parameters in Max-Pooling layer, they are **pool_size and strides**

- **Stride** : Same as the stride we introduced before, describing how many grids the pool will move in the feature map to perform max pooling.

- **Pool_size** : Pool size means the dimension of the pool, it must be a square, such as (3,3) or (4,4), hence for simplicity we often just mention the first number.
  
Here is the implementation of Max-Pooling :

In [ ]:
def maxpooling(input, size, stride):
        batch_size, in_channel, in_height, _ = input.shape[:]
        output_size = (in_height - size)//stride

        o = cp.array([
                input[:, :, i*stride:i*stride+size, j*stride:j*stride+size]
                      for i in range(output_size) for j in range(output_size)]
        ).reshape((batch_size, in_channel, output_size, output_size, size, size)).max(axis=(4,5))
#Use list comprehension to execute the maxpooling algorithm 
        assert o.shape == (batch_size, in_channel, output_size, output_size)

        return o

# Backpropagation in CNN
The backpropagation in CNN is just a more complicated version of chain rule in MLP, lets start with the backpropagation in MLP !

**1. MLP backpropagation**
 
   We can have numerous layers in MLP, but the fundamental concept behind these formulas are the same, which is the chain rule, we give the formulas **for 2 hidden layers**, you can generalize them to nth layers.
   The update formula of the weight is :

   Hence, by using chain rule of differentiation, we have:
$$\delta_3 = \frac{\partial L}{\partial Z_3} = \frac{\partial L}{\partial h_3} \cdot \frac{\partial h_3}{\partial Z_3}$$
$$\delta_2 = \frac{\partial L}{\partial Z_2} = \frac{\partial L}{\partial h_3} \cdot \frac{\partial h_3}{\partial Z_3} \cdot \frac{\partial Z_3}{\partial h_2} \cdot \frac{\partial h_2}{\partial Z_2} = \delta_3 \cdot \frac{\partial Z_3}{\partial h_2} \cdot \frac{\partial h_2}{\partial Z_2}$$
$$\delta_1 = \frac{\partial L}{\partial Z_1} = \frac{\partial L}{\partial h_3} \cdot \frac{\partial h_3}{\partial Z_3} \cdot \frac{\partial Z_3}{\partial h_2} \cdot \frac{\partial h_2}{\partial Z_2} \cdot \frac{\partial Z_2}{\partial h_1} \cdot \frac{\partial h_1}{\partial Z_1} = \delta_2 \cdot \frac{\partial Z_2}{\partial h_1} \cdot \frac{\partial h_1}{\partial Z_1}$$

By calculating the partial derivatives, we can get:
$$\delta_3 = \sigma'(Z_3)\cdot (\hat{y}-y)$$
$$\delta_2 = \delta_3 \cdot W_3^T\cdot \sigma'(Z_2)$$
$$\delta_1 = \delta_2 \cdot W_2^T \cdot \sigma'(Z_1)$$

Hence, by considering the following fact deduced from chain rule:
$$\frac{\partial L}{\partial W_3} = \frac{\partial L}{\partial Z_3}\cdot \frac{\partial Z_3}{\partial W_3} = \delta_3\cdot h_2^T$$
$$\frac{\partial L}{\partial W_2} = \frac{\partial L}{\partial Z_2}\cdot \frac{\partial Z_2}{\partial W_2} = \delta_2 \cdot h_1^T$$
$$\frac{\partial L}{\partial W_1} = \frac{\partial L}{\partial Z_1}\cdot \frac{\partial Z_1}{\partial W_1} = \delta_1 \cdot X^{T}$$

We can deduce the weight update formula for MLP:
$$W_3 \Leftarrow W_3 - \alpha \frac{\partial L}{\partial W_3}$$
$$W_2 \Leftarrow W_2 - \alpha \frac{\partial L}{\partial W_2}$$
$$W_1 \Leftarrow W_1 - \alpha \frac{\partial L}{\partial W_1}$$


In [ ]:
h=0
#Avoid auto running because the following code is just for demo, variables are not defined 
if h>0:
    error = self.output - self.one_hot
    self.delta_3 = error.T  # Shape (10, 10500)
    self.dw_3 = cp.dot(self.delta_3, self.h_2.T)  # Shape (10, 512)
    self.db_3 = cp.sum(self.delta_3, axis=1, keepdims=True)  # Shape (10, 1)
    self.delta_2 = cp.dot(self.w_3.T, self.delta_3) * self.d_relu_mlp(self.sum_2)  # Shape (512, 10500)
    self.dw_2 = cp.dot(self.delta_2, self.h_1.T)  # Shape (512, 256)
    self.db_2 = cp.sum(self.delta_2, axis=1, keepdims=True)  # Shape (512, 1)
    self.delta_1 = cp.dot(self.w_2.T, self.delta_2) * self.d_relu_mlp(self.sum_1)  # Shape (256, 10500)
    self.dw_1 = cp.dot(self.delta_1, self.output_c)  # Shape (256, 3200)
    self.db_1 = cp.sum(self.delta_1, axis=1, keepdims=True)  # Shape (256, 1)

**2. Max-Pooling backpropagation**

   For max-pooling layer, the mathematical representation of its BP is trivial, we only output gradient on the grid which is the max-pooling position, to do that, we can denote a 2-dimensional matrix called "Mask", mathematically we use the Kronecker delta as a **determinastic function**, where its value is given by:
$$\delta_{ij}=
\begin{dcases}
    1,\text{if (i,j) on max-pooling position}\\
    0,\text{otherwise}
\end{dcases}$$

Hence, we can multiply this Kronecker delta with the gradient to calculate the gradient with respect to the feature map in previous layer:

$$\frac{\partial L}{\partial x_{ij}} = \frac{\partial L}{\partial \hat{y_{ij}}} \delta_{ij}$$

In [ ]:
def maxpooling_backward(self, input, output, output_error_grad, size, stride):
        batch_size, in_channel, in_height, in_width = input.shape[:]
        output_height, output_width = output.shape[2], output.shape[3]
        input_error_grad = cp.zeros_like(input)
        for i in range(output_height):
            for j in range(output_width):
                region = input[:, :, i * stride:(i * stride) + size, j * stride:(j * stride) + size]
                max_region = cp.max(region, axis=(2, 3), keepdims=True)
                # Calculating max-pooling is easy as we demonstrated, so we simply add one line here to calculate maximum rather than calling function again.
                # We need the feature map before and after max pooling for comparison of position of maximum values
                # There is no gradient wrt to non-maximum values, the neuron who achieved maximum in max-pooling stage will gain gradient, all other neurons will get a value of zero.
                mask = (region == max_region)
                # Declare mask as a generator
                input_error_grad[:, :, i * stride:(i * stride) + size,
                j * stride:(j * stride) + size] += mask * output_error_grad[:, :, i, j][:, :, None, None]
        return input_error_grad

3. Convolution backpropagation

- The backpropagation in convolution layer is way more complex than other layers.

  (Note : The following derivation is not very rigorous)

We can use $\mathcal{H}$ and $\mathcal{K}$ to represent the collection of all points in the (q-1)th layer and qth layer:


$$\mathcal{H} = \{h^{q-1}_{ijk} : 0\leq i \leq F_{q-1} ; 0\leq j\leq F_{q-1} ;0\leq k \leq d_{q-1}\}$$
$$\mathcal{K} = \{h^{q}_{rsp}: 0\leq r \leq \frac{F_{q-1} - B_{q} +2P}{S}+1 ,s\leq \frac{F_{q-1} - B_{q} +2P}{S}+1; 0\leq p\leq d_{q}\}$$

Denote the weight tensor undergoing convolution between layer (q-1) and layer q is denoted as :

$$W^{q} = [w^{(d_{q},q)}_{B_{q},B_{q},d_{q-1}}]$$
Which means we have a weight tensor that has a shape of $B_{q-1} \times B_{q-1} \times d_{q-1}\times d_q$, which the next feature map will gain a depth of $d_q$ as we have total number of $d_q$ filters.

The basic idea is to take the derivative of (q)th layer against the kernel w:

$$\frac{\partial L}{\partial w^{q}_{mno}} = \sum_{(r,s,p)\in \mathcal{K}} \frac{\partial L}{\partial o^{q}_{rsp}} \frac{\partial o^{q}_{rsp}}{\partial h^{q}_{rsp}} \frac{\partial h^q_{rsp}}{\partial w^q_{mno}}$$

For simplicity, we use the $delta$ to represent the gradient instead:
$$\delta^{q}_{rsp}=\frac{\partial L}{\partial o^{q}_{rsp}}$$

The expression can further be simplified to :
$$\frac{\partial L}{\partial w^{q}_{mno}} = \sum_{(r,s,p)\in \mathcal{K}} \delta^{q}_{rsp} \frac{\partial o^q_{rsp}}{\partial h^q_{rsp}}\frac{\partial h^{q}_{rsp}}{\partial w^q_{mno}}$$

Referring to the previous feature map calculation, we would like to expand the formula, according to the previous formula, we can find out the $\frac{\partial h^{q}_{rsp}}{\partial w^q_{mno}}$:

$$\frac{\partial h^{q}_{rsp}}{\partial w^q_{mno}} = o^{q-1}_{rS+m-P, sS+n-P,o}$$

The partial derivative is trivial:

$$\frac{\partial L}{\partial w^q_{mno}} = \sum_{(r,s,p)\in \mathcal{K}} \sigma'(h^q_{rsp}) (\delta^{q}_{rsp} \cdot o^{q-1}_{rS+m-P, sS+n-P,o})$$

The upcoming calculation is not complex, the partial derivative against the weight is:

$$\frac{\partial L}{\partial w^q_{mno}} = \sum_{(r,s,p)\in \mathcal{K}} \sigma'(h^q_{rsp})\delta^{q}_{rsp} \cdot h^{q-1} = \sigma'(h^q_{rsp}) \text{rot}_{180} \{\delta^{q}_{rsp}\} \ast h^{q-1}$$

For the loss against the input layer, we are finding the gradient of loss against the (q-1)th layer, by using the chain rule, we can find that:

$$\frac{\partial L}{\partial h^{(q-1)}_{ijk}} = \sum_{(r,s,p)\in \mathcal{K}} \frac{\partial L}{\partial h^{q}_\mathcal{K}} \frac{\partial h^{q}_\mathcal{K}}{\partial h^{q-1}_{ijk}}$$

In this phase we are simply differentiating the successive hidden layer with respect to the previous layer, take a close look at the latter partial derivative:
$$\frac{\partial h^{q}_\mathcal{K}}{\partial h^{q-1}_{ijk}} = \frac{\partial }{\partial h^{q-1}_{ijk}} \sum_{(i,j,k)\in \mathcal{H}}W^q\cdot \sigma(h^{q-1}_{ijk})$$
$$= W^q \sigma'(h^{q-1}_{ijk})$$

We can put it back into our previous equation:
$$\frac{\partial L}{\partial h^{q+1}_{ijk}} = \sum_{(r,s,p)\in \mathcal{K}} \delta^{q}_{\mathcal{K}} W^q \sigma'(h^{q-1})$$
Again, we can flip the filter and undergo the convolution operation:
$$\frac{\partial L}{\partial h^{q+1}_{ijk}} = \text{rot}_{180}\{\sum_{(r,s,p)\in \mathcal{K}} \delta^{q}_{\mathcal{K}} W^q\} \sigma'(h^{q-1})$$
In an element-wise aspect, it can be written as :
$$\frac{\partial L}{\partial h^{q+1}_{ijk}} = \delta^{q}_{ijk}\ast \text{rot}_{180}\{ W^{q}\} \sigma'(h^{q-1}_{ijk})$$

Here is the implementation to it !

In [ ]:
def conv2d_backward(dout, X, kernel, stride, pad):
    # Dout : Upstream derivative, (N, F, out_h, out_w)
    # X : Input feature map shape, (N, C, H, W)
    # W : Kernel, (OC, IC, HH, WW)
    # stride : Number of pixels between adjacent receptive fields 
    # Pad : The number of pixels that will be used for zero padding

    N, C, H, W = X.shape 
    F, _, HH, WW = kernel.shape

    #Compute the output shape 
    out_h, out_w = (H-HH+2*pad)//stride +1, (W-WW+2*pad)//stride + 1 
    cols, k_flatten, _ = im2col_cupy(X, kernel, stride, pad)

    db = cp.sum(dout, axis=(0,2,3))

    dout_reshaped = dout.transpose(1,2,3,0).reshape(F, -1)
    cols_reshaped = cols.transpose(1,2,3,0,4,5).reshape(-1, N*out_h*out_w)

    dW = cp.matmul(dout_reshaped, cols_reshaped.T).reshape(kernel.shape)
    #Compute the gradient wrt to the input layer
    dcols = cp.matmul(k_flatten.T, dout_reshaped).reshape(N, C, HH, WW, out_h, out_w)

    dX = col2im(dcols, X, kernel, stride, pad)
    assert dX.shape == X.shape
    assert dW.shape == kernel.shape 

    return dX, dW, db

4. Program initialization

- This MNIST image recongition program refers to the structue of AlexNet (Krizhevsky, A., Sutskever, I., & Hinton, G. E. (2012)), my __init__ function is :

In [ ]:
def __init__(self, input_dim=4096, output_dim=10):

# Assuming n_1 to n_5 are the appropriate in_features for the respective layers
        self.kernel_1 = self.he_initialization(in_features=(3 * 11 * 11), shape=(96, 3, 11, 11))
        self.kernel_2 = self.he_initialization(in_features=(96 * 5 *5), shape=(256, 96, 5, 5))
        self.kernel_3 = self.he_initialization(in_features=(256 * 3 * 3), shape=(384, 256, 3, 3))
        self.kernel_4 = self.he_initialization(in_features=(384 * 3 *3), shape=(384, 384, 3, 3))
        self.kernel_5 = self.he_initialization(in_features=(384 * 3 * 3), shape=(256, 384, 3, 3))

        #He Intialization to prevent gradient explosion
        
        self.num_neurons_1 = 4096
        self.num_neurons_2 = 4096
        # AlexNet fully connected layers have 4096 neurons, hence the number of parameters in fully connected layers is 16,801,792

        self.w_1 = self.he_initialization(in_features=(input_dim), shape=(input_dim, self.num_neurons_1))
        self.w_2 = self.he_initialization(in_features=(self.num_neurons_1), shape=(self.num_neurons_1, self.num_neurons_2))
        self.w_3 = self.he_initialization(in_features=(self.num_neurons_2), shape=(output_dim, self.num_neurons_2))
        #Debugging remark : self.w_1 - self.w_3 initialized weights are around 9e-03 - 9e-05, acceptable

        self.b_1 = cp.zeros((self.num_neurons_1, 1))
        self.b_2 = cp.zeros((self.num_neurons_2, 1))
        self.b_3 = cp.zeros((output_dim, 1))

        # Defining the filters, neurons and biases
        if not hasattr(self, 'v_1'):
            self.v_1 = cp.zeros_like(self.w_1)
            self.v_2 = cp.zeros_like(self.w_2)
            self.v_3 = cp.zeros_like(self.w_3)

        self.d_1 = cp.zeros_like(self.w_1)
        self.d_2 = cp.zeros_like(self.w_2)
        self.d_3 = cp.zeros_like(self.w_3)

        self.a_1 = cp.zeros_like(self.kernel_1)
        self.a_2 = cp.zeros_like(self.kernel_2)
        self.a_3 = cp.zeros_like(self.kernel_3)
        self.a_4 = cp.zeros_like(self.kernel_4)
        self.a_5 = cp.zeros_like(self.kernel_5)

        self.c_1 = cp.zeros_like(self.kernel_1)
        self.c_2 = cp.zeros_like(self.kernel_2)
        self.c_3 = cp.zeros_like(self.kernel_3)
        self.c_4 = cp.zeros_like(self.kernel_4)
        self.c_5 = cp.zeros_like(self.kernel_5)

        self.learning_rate = 0.001
        self.epsilon = 1e-8
        self.beta_1 = 0.9
        self.beta_2 = 0.99
        self.epochs = 60
        self.batch_size = 32
        self.l_accuracy = []
        self.l_loss = []
        self.loss_values = []
        self.accuracy_values = []

    # Intialization part
    # Definitng the activation functions for the convolution and MLP

def he_initialization(self, in_features, shape): 
    limit = cp.sqrt(2 / in_features) 
    return cp.random.uniform(low=-limit, high=limit, size=shape)

5. Besides, we also need to declare activation functions, which are : 

In [ ]:
def one_hot_encoder(self, label):
    self.one_hot = cp.zeros((label.size, 10))
    self.one_hot[cp.arange(label.size), label] = 1
    #One-Hot encoding
    return self.one_hot

    # One-Hot encoder to turn label to shape of (6,6)
    @staticmethod
    def relu(input):
        if input is None:
            raise TypeError("Input of relu activation is None")
        return cp.maximum(0, input)
        # The formal definition of relu 

    @staticmethod
    def d_relu(output, grad_output):
        if output is None or grad_output is None:
            raise TypeError("Output or grad_output of d_relu is None")
        relu_grad = (output > 0).astype(cp.float32)
        # Why use a generator here? Because it will output 1(True) and 0(False), exactly same as derivative of relu !
        return grad_output*relu_grad
        '''Why we need the grad_output to compare is because, when we are using d_relu, its already in backprop phase, hence,
        we need to know the output of relu first, then we can know which value (1 or 0) to output.
        '''

    @staticmethod
    def d_relu_mlp(input):
        return cp.where(input > 0, 1, 0)

    @staticmethod
    def softmax(x):
        exp_x = cp.exp(x - cp.max(x, axis=0, keepdims=True))
        return exp_x / cp.sum(exp_x, axis=0, keepdims=True)

We also give the definitions of the activation functions we use here :

- ReLU(Rectifier) :
  $$ReLU(x) = \begin{dcases}
    x,&,\text{if}  x\geq 0\\
    0,&,\text{if}  x<0
\end{dcases}$$

- Softmax :
  $$\sigma(x_n) = \frac{e^{x_n}}{\sum^{n}_{i=1} e^{x_i}}$$

6. **Adam Optimizer**
- Adam optimization is proposed by Kingma, D. P., & Ba, J. (2014), which is one of the most effective optimization algorithm in the world, the reason is it combined momentum learning method(Polyak, B. T., 1964) and RMSProp(Tieleman, T., & Hinton, G. (2012))

* Momentum method :

 We understand that the goal of deep learning is simply **finding a weight tensor such that the loss function of this weight is lowest**, hence, instead of directly updating the weight, we can instead store, aggregate and weight them :

  $$V \Leftarrow -\alpha \frac{\partial L}{\partial W}$$
  $$V \Leftarrow \beta V - \alpha \frac{\partial L}{\partial W}$$
  $$W \Leftarrow W+V$$

* RMSProp Method :

  The RMSProp weight the squared gradient using a decay factor, and square root to normalize the aggregated gradeints :

  $$A_i \Leftarrow \rho A_i + (1-\rho)(\frac{\partial L}{\partial w_i})^2 \forall i $$
  $$w_i \Leftarrow w_i - \frac{\alpha}{\sqrt{A_i}} (\frac{\partial L}{\partial w_i})^2$$

  One of the advantage of RMSProp is that the gradient will decay over time

* Adam Method:

  After we understood RMSProp and Momentum, we can move to Adam, Adam(Adaptive moment) combines both momentum and RMSProp, in Adam, we have first order and second order moment, which is $F_i$ and $A_i$, for the second order moment $A_i$ :
  $$A_i \Leftarrow \rho A_i + (1-\rho)(\frac{\partial L}{\partial w_i})^2$$

  For first order moment, we have another decay rate $\rho_f$:
  $$F_i \Leftarrow \rho_f F_i + (1-\rho_f)(\frac{\partial L}{\partial w_i})$$

The weight update formula becomes : 
$$w_i \Leftarrow w_i - \alpha_t \frac{F_i}{\sqrt{A_i}}$$

Where:

$$\alpha _t = \alpha \frac{\sqrt{1-\rho^t} }{1-\rho_{f}^t}$$

The reason to update the learning rate is to **correct bias**, biases often appear when our training data is not diverse enough or the model is overfitting. In Adam, we initialized $A_i$ and $F_i$ to 0, so it must have caused some biases initially.



In [ ]:
def update_params(self, t):
    # Update for CNN kernels using Adam, where a_1 means the first order moment, c_1 means second order moment, t = iterations
    self.a_1 = self.beta_1 * self.a_1 + (1 - self.beta_1) * self.dw_conv_1
    self.c_1 = self.beta_2 * self.c_1 + (1 - self.beta_2) * cp.square(self.dw_conv_1)
    m_hat_1 = self.a_1 / (1 - cp.power(self.beta_1, t) + self.epsilon)
    v_hat_1 = self.c_1 / (1 - cp.power(self.beta_2, t) + self.epsilon)

    self.kernel_1 -= self.learning_rate * m_hat_1 / (cp.sqrt(v_hat_1) + epsilon)
    #Updating kernel in Convolution layer 

    self.v_1 = self.beta_1 * self.v_1 + (1 - self.beta_1) * self.dw_1
    self.d_1 = self.beta_2 * self.d_1 + (1 - self.beta_2) * cp.square(self.dw_1)
    m_hat_w1 = self.v_1 / (1 - cp.power(self.beta_1, t) + self.epsilon)
    v_hat_w1 = self.d_1 / (1 - cp.power(self.beta_2, t) + self.epsilon)
    #Updating weights in MLP layer 
    
    self.w_1 -= self.learning_rate * m_hat_w1 / (cp.sqrt(v_hat_w1) + epsilon)
    #The update logic in other layers is exactly the same, so we only demonstrate the update of one layer for your reference

7. **Batch Normalization(Optional)**

Here we refer to C.C.Aggarwal[1, Sec 4.8], we can break up the addition and activation into pre-activation and post-activation, the batch normalization simply add a normalization layer in between them to normalize the input of batch elements, which can prevent many common problems such as exploding and vanishing gradient problems, first, we need to calculate the mean and standard deviation, assume we have m instances in a batch:

$$\mu_i = \frac{\sum^{m}_{r=1} v^{(r)}_i}{m} \forall i$$
$$\sigma^2_{i} = \frac{\sum^{m}_{r=1} (v^(r)_i - \mu_i)^2}{m} + \varepsilon \forall i$$

The new batch element value is given by:
$$\hat{v}^{r}_i = \frac{v^(r)_i - \mu_i}{\sigma_i} \forall i,r$$

The pre-activation value is given by:
$$a^{(r)}_i = \gamma_i \cdot \hat{v^(r)_i} + \beta_i \forall i,r$$

We also give the formula for the backpropagation here in order to ensure mathematical rigour, the keypoint of batch-normalization is that we also need to update the parameter $\beta_i$ and $\gamma_i$ for optimal performance, this can be done by using gradient descent, by chain rule, it is apparent that:

$$\frac{\partial L}{\partial \beta_i} = \sum^{m}_{r=1} \frac{\partial L}{\partial a^{(r)}_i} \cdot \frac{\partial a^{(r)}_i}{\partial \beta_i} 
= \sum^{m}_{r=1} \frac{\partial L}{\partial a^{(r)}_i}$$

$$\frac{\partial L}{\partial \gamma_i} = \sum^{m}_{r=1} \frac{\partial L}{\partial a^{(r)}_i} \cdot \frac{\partial a^{(r)}_i}{\partial \gamma_i} 
= \sum^{m}_{r=1} \frac{\partial L}{\partial a^{(r)}_i} \cdot \hat{v^(r)_i}$$
\newpage 
Our objective is to calculate $\frac{\partial L}{\partial v^{(r)}}_i$ for further back-propagation, therefore, we need to calculate the 3 partial derivatives, the derivations are trivial, hence we give the formula directly:

$$\frac{\partial L}{\partial v_i^{(r)}} = 
\frac{\partial L}{\partial a_i^{(r)}} \left( \frac{\gamma_i}{\sigma_i} \right) + \frac{\partial L}{\partial \mu_i} \left( \frac{1}{m} \right) + \frac{\partial L}{\partial \sigma_i^2} \left( \frac{2(v_i^{(r)} - \mu_i)}{m} \right)$$

$$\frac{\partial L}{\partial \sigma_i^2} = \sum_{q=1}^{m} \frac{\partial L}{\partial \hat{v}_i^{(q)}} \cdot \frac{\partial \hat{v}_i^{(q)}}{\partial \sigma_i^2} $$

$$= -\frac{1}{2\sigma_i^3} \sum_{q=1}^{m} \frac{\partial L}{\partial \hat{v}_i^{(q)}} (v_i^{(q)} - \mu_i) $$
$$= -\frac{1}{2\sigma_i^3} \sum_{q=1}^{m} \frac{\partial L}{\partial a_i^{(q)}} \gamma_i \cdot (v_i^{(q)} - \mu_i)
$$


$$\frac{\partial L}{\partial \mu_i} = \sum_{q=1}^{m} \frac{\partial L}{\partial \hat{v}_i^{(q)}} \cdot \frac{\partial \hat{v}_i^{(q)}}{\partial \mu_i} + \frac{\partial L}{\partial \sigma_i^2} \cdot \frac{\partial \sigma_i^2}{\partial \mu_i} = $$

$$-\frac{1}{\sigma_i} \sum_{q=1}^{m} \frac{\partial L}{\partial \hat{v}_i^{(q)}} - 2 \frac{\partial L}{\partial \sigma_i^2} \cdot \frac{\sum_{q=1}^{m} (v_i^{(q)} - \mu_i)}{m} $$

$$= -\frac{\gamma_i}{\sigma_i} \sum_{q=1}^{m} \frac{\partial L}{\partial a_i^{(q)}} + $$

$$\left( \frac{1}{\sigma_i^3} \cdot \left( \sum_{q=1}^{m} \frac{\partial L}{\partial a_i^{(q)}} \gamma_i \cdot (v_i^{(q)} - \mu_i) \right) \right) \cdot \left( \frac{\sum_{q=1}^{m} (v_i^{(q)} - \mu_i)}{m} \right)
$$

The following code is a simple implementation to batch normalization and its backpropagation:

In [ ]:
class BatchNormalization:
    def __init__(self, num_features, epsilon=1e-5, momentum=0.9):
        self.epsilon = epsilon
        self.momentum = momentum
        self.gamma = cp.ones(num_features)
        self.beta = cp.zeros(num_features)
        self.running_mean = cp.zeros(num_features)
        self.running_var = cp.ones(num_features)

    def forward(self, x, training=True):
        if training:
            batch_mean = cp.mean(x, axis=0)
            batch_var = cp.var(x, axis=0)
            self.x_centered = x - batch_mean
            #We will always use the (v-mean) in backward phase, so define it first
            self.stddev_inv = 1.0 / cp.sqrt(batch_var + self.epsilon)
            #Add epsilon to prevent division by 0 
            #stddev_inv = inverse of standard deviation 
            x_norm = self.x_centered * self.stddev_inv
            self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * batch_mean
            self.running_var = self.momentum * self.running_var + (1 - self.momentum) * batch_var
        else:
            x_norm = (x - self.running_mean) / cp.sqrt(self.running_var + self.epsilon)

        self.out = self.gamma * x_norm + self.beta
        return self.out

    def backward(self, dout):
        N, D = dout.shape

        x_norm = self.x_centered * self.stddev_inv
        dbeta = cp.sum(dout, axis=0)
        dgamma = cp.sum(dout * x_norm, axis=0)

        dx_norm = dout * self.gamma
        dvar = cp.sum(dx_norm * self.x_centered * -0.5 * self.stddev_inv**3, axis=0)
        dmean = cp.sum(dx_norm * -self.stddev_inv, axis=0) + dvar * cp.mean(-2.0 * self.x_centered, axis=0)

        dx = (dx_norm * self.stddev_inv) + (dvar * 2.0 * self.x_centered / N) + (dmean / N)

        return dx, dgamma, dbeta

You can integrate it into the CNN on your own !

In [ ]:
import cupy as cp 
import cupyx.distributed.array as cda 
import pandas as pd
import time 
import numpy as np 
import sys 
import gc
gc.enable()

path = "/kaggle/input/digit-recognizer/train.csv"
path1 = "/kaggle/input/digit-recognizer/test.csv"
data = pd.read_csv(path, engine='c')
test_data = pd.read_csv(path1, engine='c')
test_data = test_data/255.0
test_data = cp.array(test_data).reshape(28000, 1, 28, 28)
label_1 = data['label']
label_1 = label_1.T

data = cp.array(data)
m,n = data.shape
train_data = data[0:m].T
X_train = train_data[1:n]
X_train = X_train/255.0
X_train = X_train.reshape(42000, 1, 28, 28)


def one_hot_encoding(label):
    one_hot = cp.zeros((label.size, 10))
    one_hot[cp.arange(label.size), label]=1
    return one_hot 

class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights and biases
        self.W1 = cp.random.randn(input_size, hidden_size) * cp.sqrt(2. / input_size)
        self.b1 = cp.zeros(hidden_size)
        self.W2 = cp.random.randn(hidden_size, hidden_size) * cp.sqrt(2. / hidden_size)
        self.b2 = cp.zeros(hidden_size)
        self.W3 = cp.random.randn(hidden_size, output_size) * cp.sqrt(2. / hidden_size)
        self.b3 = cp.zeros(output_size)

    def relu(self, x):
        return cp.maximum(0, x)

    def relu_backward(self, dout, cache):
        dx = dout.copy()
        dx[cache <= 0] = 0
        return dx 
    
    def softmax(self,x):
        e = cp.exp(x-cp.max(x, axis=1, keepdims=True))
        return e/cp.sum(e, axis=1, keepdims=True)

    def cross_entropy_loss(self, y_pred, y_true):
        m = y_true.shape[0]
        L = -cp.sum(y_true*cp.log(y_pred+1e-15), axis=1)
        loss = cp.mean(L)
        return loss 
    
    def forward(self, X):
        # Forward pass
        self.z1 = X.dot(self.W1) + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = self.a1.dot(self.W2) + self.b2
        self.a2 = self.relu(self.z2)
        self.z3 = self.a2.dot(self.W3) + self.b3
        self.a3 = self.softmax(self.z3)  # Assuming no activation on the output layer (e.g., for regression)

        return self.a3

    def backward(self, X, y, shape, learning_rate=1e-3):
        # Backward pass
        m = y.shape[0]

        # Assuming mean squared error loss
        dz3 = (self.a3-y)/m 

        dW3 = self.a2.T.dot(dz3)
        db3 = cp.sum(dz3, axis=0)

        da2 = dz3.dot(self.W3.T)
        dz2 = self.relu_backward(da2, self.z2)
        dW2 = self.a1.T.dot(dz2)
        db2 = cp.sum(dz2, axis=0)

        da1 = dz2.dot(self.W2.T)
        dz1 = self.relu_backward(da1, self.z1)
        dW1 = X.T.dot(dz1)
        db1 = cp.sum(dz1, axis=0)

        self.deflatten = cp.matmul(da1, self.W1.T).reshape(shape)

        # Update weights and biases
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W3 -= learning_rate * dW3
        self.b3 -= learning_rate * db3

def distributed_matmul(A: cp.array, B: cp.array):
    """
    Performs distributed matrix multiplication of A (MxK) and B (KxN) using two GPUs.
    Splits A into two parts along rows and multiplies each part with B on separate GPUs.
    Combines the results to form the final (MxN) matrix.

    More assertions are made to avoid illegal inputs.
    """
    M, K = A.shape
    _, N = B.shape

    # Ensure inputs are valid
    assert A is not None and B is not None, "Input matrices cannot be None"
    assert K == B.shape[0], "Matrix dimensions do not match for multiplication"
    assert cp.cuda.Device(0) is not None and cp.cuda.Device(1) is not None, "No GPU on your computer or GPU is not detected"

    # Split matrix A into two parts along rows
    split_M = M // 2
    A1 = A[:split_M, :]  # First half of A (split_M x K)
    A2 = A[split_M:, :]  # Second half of A (M - split_M x K)

    # Create distributed arrays for A1 and A2
    a1 = cda.distributed_array(A1, index_map={0: (slice(0, split_M), slice(0, K))})
    a2 = cda.distributed_array(A2, index_map={1: (slice(0, M - split_M), slice(0, K))})

    # Create distributed array for B (shared across GPUs)
    b = cda.distributed_array(B, index_map={0: (slice(0, K), slice(0, N)), 1:(slice(0, K), slice(0, N))})

    # Perform matrix multiplication on GPU 0
    C1,C2 = cda.matmul(a1, b), cda.matmul(a2,b)  # C1 = A1 @ B (split_M x N)
    C1_local = cp.array(C1.get(order='C')) # Collect result to local memory
    C2_local = cp.array(C2.get(order='C'))

    del a1, a2, b, C1, C2

    # Concatenate C1 and C2 along rows to form the final result
    C = cp.vstack((C1_local, C2_local))  # C = [C1; C2] (M x N)
    
    # Clean up distributed arrays
    del C1_local, C2_local

    # Verify the final shape
    assert C.shape == (M, N), f"Matrix multiplication error: Expected shape {(M, N)}, got {C.shape}"
    return C

def im2col_cupy(X_train:cp.array, kernel:cp.array, stride:int, pad:int) -> tuple:
    # Flatten the kernel
    k_flatten = cp.reshape(kernel, newshape=(kernel.shape[0], -1))

    # Calculate output shape
    N, C, H, W = X_train.shape
    OC, IC, HH, WW = kernel.shape
    out_h, out_w = (H - HH + 2 * pad) // stride + 1, (W - WW + 2 * pad) // stride + 1

    # Pad the input
    if pad > 0:
        X_padded = cp.pad(X_train, pad_width=[(0, 0), (0, 0), (pad, pad), (pad, pad)], mode='constant')
    else:
        X_padded = X_train

    # Generate column matrix using as_strided
    columns = cp.lib.stride_tricks.as_strided(
        X_padded,
        shape=(N, C, out_h, out_w, HH, WW),
        strides=(
            X_padded.strides[0],
            X_padded.strides[1],
            X_padded.strides[2] * stride,
            X_padded.strides[3] * stride,
            X_padded.strides[2],
            X_padded.strides[3]
        )
    ).reshape(C*HH*WW, N*out_h*out_w)

    # Perform matrix multiplication
    a = distributed_matmul(k_flatten, columns).reshape(N,OC,out_h,out_w)

    return columns.reshape(N, C, HH, WW, out_h, out_w), k_flatten, a

def col2im(cols:cp.array, input:cp.array, kernel:cp.array, stride:int, pad:int) -> cp.array:

    N, C, H, W = input.shape
    _, _, HH, WW = kernel.shape 
    out_h = (H+2*pad-HH)//stride + 1
    out_w = (W+2*pad-WW)//stride + 1
    cols = cols.reshape(N*C, HH*WW, out_h, out_w)
    X = cp.zeros((N,C,H+2*pad,W+2*pad))
    cols : cp.arary 
    X : cp.array 

    #Place an assertion to detect incompitable shapes early 

    for i in range(HH): 
        for j in range(WW): 
            X[:, :, i:i+stride*out_h:stride, j:j+stride*out_w:stride] += cols[:, i*WW+j, :, :].reshape(N, C, out_h, out_w)

    if pad>0:
        X = X[:, :, pad:-pad, pad:-pad]
    return X 

def conv2d_backward(dout, X, kernel, stride, pad):

    # Dout : Upstream derivative, (N, F, out_h, out_w)
    # X : Input feature map shape, (N, C, H, W)
    # W : Kernel, (OC, IC, HH, WW)
    # stride : Number of pixels between adjacent receptive fields 
    # Pad : The number of pixels that will be used for zero padding

    N, C, H, W = X.shape 
    F, _, HH, WW = kernel.shape

    #Compute the output shape 
    out_h, out_w = (H-HH+2*pad)//stride +1, (W-WW+2*pad)//stride + 1 
    cols, k_flatten, _ = im2col_cupy(X, kernel, stride, pad)

    db = cp.sum(dout, axis=(0,2,3))

    dout_reshaped = dout.transpose(1,2,3,0).reshape(F, -1)
    cols_reshaped = cols.transpose(1,2,3,0,4,5).reshape(-1, N*out_h*out_w)
    
    dW = distributed_matmul(dout_reshaped, cols_reshaped.T).reshape(kernel.shape)
    #Compute the gradient wrt to the input layer
    dcols = distributed_matmul(k_flatten.T, dout_reshaped).reshape(N, C, HH, WW, out_h, out_w)

    dX = col2im(dcols, X, kernel, stride, pad)
    assert dX.shape == X.shape
    assert dW.shape == kernel.shape 

    return dX, dW, db

class CNN:
    def __init__(self, input_shape, num_classes):
        # Initialize layers
        self.conv_kernel = cp.random.randn(8, input_shape[1], 3, 3) * cp.sqrt(2. / (3 * 3 * input_shape[1]))
        self.conv_stride = 1
        self.conv_pad = 1
        
        self.pool_size = 2
        self.pool_stride = 2
        
        # Calculate the size of the flattened layer after pooling
        conv_output_height = (input_shape[2] - 3 + 2 * self.conv_pad) // self.conv_stride + 1
        conv_output_width = (input_shape[3] - 3 + 2 * self.conv_pad) // self.conv_stride + 1
        pool_output_height = (conv_output_height - self.pool_size) // self.pool_stride + 1
        pool_output_width = (conv_output_width - self.pool_size) // self.pool_stride + 1
        
        flattened_size = 8 * pool_output_height * pool_output_width
        
        # Initialize MLP
        self.mlp = MLP(flattened_size, 100, num_classes)

    def relu(self, x):
        return cp.maximum(0, x)

    def relu_backward(self, dout, cache):
        dx = dout.copy()
        dx[cache <= 0] = 0
        return dx 
    
    def forward(self, X):
        # Convolutional layer
        self.conv_out, self.conv_cols, self.a = im2col_cupy(X, self.conv_kernel, self.conv_stride, self.conv_pad)
        self.a_1 = self.relu(self.a)
        
        # Max pooling layer
        self.pool_out = self.maxpool_forward(self.a)
        self.flat_shape = self.pool_out.shape 
        # Flatten
        self.flattened = self.pool_out.reshape(X.shape[0], -1)
        
        # Fully connected layer
        output = self.mlp.forward(self.flattened)
        
        return output
    
    def backward(self, X, y, learning_rate=1e-3):
        # Backward pass through MLP
        self.mlp.backward(self.flattened, y, shape=self.flat_shape, learning_rate=learning_rate)
        
        # Backward pass through max pooling
        pool_error_grad = self.mlp.deflatten
        conv_error_grad = self.maxpool_backward(self.a, self.pool_out, pool_error_grad)
        
        # Backward pass through convolutional layer
        conv_error_grad_relu = self.relu_backward(conv_error_grad, self.a_1)
        dX, dW, db = conv2d_backward(conv_error_grad_relu, X, self.conv_kernel, self.conv_stride, self.conv_pad)
        
        # Update convolutional kernel
        self.conv_kernel -= learning_rate * dW
    
    def maxpool_forward(self, input):
        batch_size, in_channel, in_height, in_width = input.shape
        output_height = (in_height - self.pool_size) // self.pool_stride + 1
        output_width = (in_width - self.pool_size) // self.pool_stride + 1
        
        output = cp.array([
            input[:, :, i * self.pool_stride:i * self.pool_stride + self.pool_size, j * self.pool_stride:j * self.pool_stride + self.pool_size]
            for i in range(output_height) for j in range(output_width)
        ]).reshape((batch_size, in_channel, output_height, output_width, self.pool_size, self.pool_size)).max(axis=(4, 5))
        
        return output
    
    def maxpool_backward(self, input, output, output_error_grad):
        batch_size, in_channel, in_height, in_width = input.shape
        output_height, output_width = output.shape[2], output.shape[3]
        input_error_grad = cp.zeros_like(input)
        
        for i in range(output_height):
            for j in range(output_width):
                region = input[:, :, i * self.pool_stride:(i * self.pool_stride) + self.pool_size, j * self.pool_stride:(j * self.pool_stride) + self.pool_size]
                max_region = cp.max(region, axis=(2, 3), keepdims=True)
                mask = (region == max_region)
                input_error_grad[:, :, i * self.pool_stride:(i * self.pool_stride) + self.pool_size,
                                 j * self.pool_stride:(j * self.pool_stride) + self.pool_size] += mask * output_error_grad[:, :, i, j][:, :, None, None]
        
        return input_error_grad
    
    def train(self, X, y, epochs=10, learning_rate=1e-3, batch_size=21000):
        for epoch in range(epochs):
            for batch in range(42000//batch_size):
                start = batch_size*batch
                end = start+batch_size
                X_batch = X[start:end]
                label_batch = y[start:end]
                output = self.forward(X_batch)
                self.backward(X_batch, label_batch, learning_rate)
                loss = self.mlp.cross_entropy_loss(output, label_batch)
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss}')

    def test(self, X):
        r = cp.argmax(self.forward(X), axis=1)
        r = cp.asnumpy(r)
        submission = pd.DataFrame({
        'ImageId': range(1, len(r) + 1),
        'Label': r
        })

# Save the DataFrame to a CSV file
        submission.to_csv("/kaggle/working/submission.csv", index=False)

cnn = CNN(X_train.shape, 10)

one_hot_encoded_y = one_hot_encoding(label_1)
cnn.train(X_train, one_hot_encoded_y, epochs=100, learning_rate=0.3)
cnn.test(test_data)

Epoch 1/100, Loss: 2.5409883113930563
Epoch 2/100, Loss: 2.3535367669833165
Epoch 3/100, Loss: 2.3159065606456406
Epoch 4/100, Loss: 2.3128064625246907
Epoch 5/100, Loss: 2.310843676816196
Epoch 6/100, Loss: 2.309351068161322
Epoch 7/100, Loss: 2.3082192018056205
Epoch 8/100, Loss: 2.3072247527143928
Epoch 9/100, Loss: 2.306350403062044
Epoch 10/100, Loss: 2.3056198899065117
Epoch 11/100, Loss: 2.3049593067248435
Epoch 12/100, Loss: 2.3043398131184483
Epoch 13/100, Loss: 2.3037810599619006
Epoch 14/100, Loss: 2.303253460609865
Epoch 15/100, Loss: 2.302753656603038
Epoch 16/100, Loss: 2.3022701428516945
Epoch 17/100, Loss: 2.3018061174567412
Epoch 18/100, Loss: 2.3013430110141595
Epoch 19/100, Loss: 2.3009416356002887
Epoch 20/100, Loss: 2.3004974652329455
Epoch 21/100, Loss: 2.3000772749253326
Epoch 22/100, Loss: 2.2996842787218075
Epoch 23/100, Loss: 2.2993072899833003
Epoch 24/100, Loss: 2.2989276828935377
Epoch 25/100, Loss: 2.298543856429749
Epoch 26/100, Loss: 2.29813073169519
Epo

References :
1. Hubel, D. H., & Wiesel, T. N. (1959a). Receptive fields of single neurons in the cat’s striate cortex. Retrieved from https://pmc.ncbi.nlm.nih.gov/articles/PMC1363130/pdf/jphysiol01298-0128.pdf
2. He, K., Zhang, X., Ren, S., & Sun, J. (2015a). Delving deep into rectifiers: Surpassing human-level performance on ImageNet Classification. Retrieved from https://arxiv.org/abs/1502.01852
3. Chellapilla, K., Puri, S., & Simard, P. (2006). High performance convolutional neural networks for document processing. Proceedings of the International Workshop on Frontiers in Handwriting Recognition (IWFHR-10).
4. Kine, J. (2016, September 5). Backpropagation in Convolutional Neural Networks. Retrieved from https://www.jefkine.com/general/2016/09/05/backpropagation-in-convolutional-neural-networks/
5. Aggarwal, C. C. (2018). Neural networks and deep learning: A textbook. Springer.
6. Krizhevsky, A., Sutskever, I., & Hinton, G. E. (2012). Imagenet classification with deep convolutional neural networks. In Advances in Neural Information Processing Systems (pp. 1097-1105).
7. Kingma, D. P., & Ba, J. (2014). Adam: A Method for Stochastic Optimization. Retrieved from https://arxiv.org/abs/1412.6980
8. Tieleman, T., & Hinton, G. (2012). Lecture 6.5—Rmsprop: Divide the gradient by a running average of its recent magnitude.
9. Polyak, B. T. (1964). Some methods of speeding up the convergence of iteration methods. Journal of Applied Mathematics and Mechanics, 28(5), 984-993.
10. S. Ioffe and C. Szegedy, “Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift,” arXiv.org, 2015. https://arxiv.org/abs/1502.03167
‌